In [1]:
!pip install kaggle

Dependencies

In [2]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings("ignore")

Data collection - Kaggle API

In [3]:
kaggle_dictionary = json.load(open('kaggle.json'))
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [4]:
print(kaggle_dictionary)

{'username': 'pujithagali', 'key': '3d3a1b242c6e93da93dd4974ab7c6af9'}


In [5]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other


In [6]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [7]:
#unzip the dataset
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip','r') as zip_ref:
  zip_ref.extractall()


In [8]:
data = pd.read_csv('IMDB Dataset.csv')

In [9]:
data.shape

(50000, 2)

In [10]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [12]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


one-hot encodeing  or label encoding

In [13]:
#positive -> 1
#negative -> 0
data.replace({'sentiment':{'positive':1,'negative':0}},inplace=True)

In [14]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [15]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [16]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


LSTM -> Long short term memory

In [17]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)

In [18]:
train_data.shape

(40000, 2)

In [19]:
test_data.shape

(10000, 2)

In [20]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [21]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [22]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [23]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [24]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [25]:
Y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


LSTM Model Building

In [26]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.build(input_shape=(None, 200))

In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [29]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 210s 405ms/step - accuracy: 0.7208 - loss: 0.5375 - val_accuracy: 0.8396 - val_loss: 0.3747
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 263s 415ms/step - accuracy: 0.8413 - loss: 0.3801 - val_accuracy: 0.8413 - val_loss: 0.3750
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 262s 416ms/step - accuracy: 0.8633 - loss: 0.3399 - val_accuracy: 0.8579 - val_loss: 0.3457
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 210s 420ms/step - accuracy: 0.8870 - loss: 0.2830 - val_accuracy: 0.8665 - val_loss: 0.3261
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 262s 420ms/step - accuracy: 0.8973 - loss: 0.2607 - val_accuracy: 0.8581 - val_loss: 0.3422


In [30]:
model.save("model.keras")

In [31]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [32]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 117ms/step - accuracy: 0.8540 - loss: 0.3424


In [33]:
print(loss)

0.3376125693321228


In [34]:
print(accuracy)

0.8583999872207642


Building Predictive System


In [35]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [36]:
predictive_system("This movie was fantastic and amazing")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step


'positive'

In [37]:
predictive_system("A trilling adventure with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


'positive'

In [38]:
predictive_system("A visual masterpiece")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step


'positive'

In [39]:
predictive_system("Overall long and slow")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step


'negative'

In [44]:
predictive_system("boring")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step


'negative'